In [16]:
import nltk
from nltk.corpus import wordnet as wn
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse.csgraph import floyd_warshall, connected_components
import operator
from collections import defaultdict
import numpy as np
import networkx as nx
import json
from collections import defaultdict

In [17]:
# Some definitions

# Reflection (circle inversion of x through orthogonal circle centered at a)
def isometric_transform(a, x):
    r2 = np.linalg.norm(a)**2 - (1.0)
    return r2/np.linalg.norm(x - a)**2 * (x-a) + a

# Inversion taking mu to origin
def reflect_at_zero(mu,x):
    a = mu/np.linalg.norm(mu)**2
    return isometric_transform(a,x)

def acosh(x):
    return np.log(x + np.sqrt(x**2-1))

# Hyperbolic distance
def dist(u,v):
    z  = 2 * np.linalg.norm(u-v)**2
    uu = 1. + z/((1-np.linalg.norm(u)**2)*(1-np.linalg.norm(v)**2))
    return acosh(uu)

# Hyperbolic distance from 0
def hyp_dist_origin(x):
    return np.log((1+np.linalg.norm(x))/(1-np.linalg.norm(x)))

# Scalar multiplication w*x
def hyp_scale(w, x):
    sgn = (-1.0)**float(w<0)
    w *= sgn    
    if w == 1:
        return sgn*x
    else:
        x_dist = (1+np.linalg.norm(x))/(1-np.linalg.norm(x))
        alpha = 1-2/(1+x_dist**w)
        alpha *= 1/np.linalg.norm(x)
    
    return sgn*alpha*x

# Convex combination (1-w)*x+w*y
def hyp_conv_comb(w, x, y):
    # circle inversion sending x to 0
    (xinv, yinv) = (reflect_at_zero(x, x), reflect_at_zero(x, y))
    # scale by w 
    pinv = hyp_scale(w, yinv)
    # reflect back
    return reflect_at_zero(x, pinv)

# Weighted sum w1*x + w2*y
def hyp_weighted_sum(w1, w2, x, y):
    p = hyp_conv_comb(w2 / (w1 + w2), x, y)
    return hyp_scale(w1 + w2, p)

In [18]:
# Creating the edge list for hypernym relationship in Wordnet.

def load_wordnet():
    SynstoIDs  = dict()
    IDstoSyns = dict()
    all_syns = list(wn.all_synsets())
    
    for idx, x in enumerate(all_syns):
        SynstoIDs[x] = idx
        IDstoSyns[idx] = x

    n = len(all_syns)
    e = make_edge_set()

    for idx, x in enumerate(all_syns):
        for y in x.hypernyms():
            y_idx = SynstoIDs[y]
            add_edge(e, idx  , y_idx)
            add_edge(e, y_idx,   idx)
            
    X = csr_matrix(e, shape=(n, n))

    return SynstoIDs, IDstoSyns, X, all_syns

    
SynstoIDs, IDstoSyns, X, all_syns = load_wordnet()
G = nx.from_scipy_sparse_matrix(X)
Gc = max(nx.connected_component_subgraphs(G), key=len)

# reorder with nx
Gc_final = nx.convert_node_labels_to_integers(Gc, ordering="decreasing degree", label_attribute="old_label")

#Create the dict for old-id <-> new-id matching for syns
RefDict = Gc_final.node
IDsToSyns_f = dict()
SynsToIDs_f = dict()
for new_idx in RefDict.keys():
    old_idx = RefDict[new_idx]['old_label']
    curr_syn = IDstoSyns[old_idx]
    IDsToSyns_f[new_idx] = curr_syn
    SynsToIDs_f[curr_syn] = new_idx
    


In [19]:
# Read the emb files, save their tau and emb_dict.

emb_files = {
            'hypernym_demo.emb':'hypernym_demo',
            }

RelEmbDict = defaultdict(dict)
RelTauDict = defaultdict(dict)
for file in emb_files.keys():
    with open(file, 'r') as emb:
        emb_lines = emb.readlines()    
    emb_lines = emb_lines[1:]
    emb_dict = dict()
    rel = emb_files[file]
    for idx, line in enumerate(emb_lines):
        curr_line = line.split(',')
        curr_tau = curr_line[-1].split("\n")[0]
        curr_tau = np.float64(curr_tau)
        curr_line = curr_line[:-1]
        curr_idx = int(curr_line[0])                
        emb_dict[curr_idx] = np.asarray(list(map(np.float64, curr_line[1:])))
    RelEmbDict[rel] = emb_dict
    RelTauDict[rel] = curr_tau
    

#Construct W matrices for each relationship separately. 
    
vector_dim = 10
ReltoW = defaultdict()
for rel in RelEmbDict.keys():
    emb_dict_curr = RelEmbDict[rel]
    vocab_size = len(emb_dict_curr)
    W_curr = np.zeros((vocab_size, vector_dim))
    for idx, vec in emb_dict_curr.items():
        W_curr[idx,:] = vec
    ReltoW[rel] = W_curr
    


In [20]:
# Find the top 10 nearest neighbors to a particular synset for given relationship.

vector_dim = 10
rel = 'hypernym_demo'
emb_dict_curr = RelEmbDict[rel]
vocab_size = len(emb_dict_curr)
W = np.zeros((vocab_size, vector_dim))
relTau = RelTauDict[rel]


e1 = wn.synset('geometry.n.01')
e1_idx = SynsToIDs_f[e1]


for idx, vec in emb_dict_curr.items():
    W[idx,:] = vec
    
vec_e1 = emb_dict_curr[e1_idx] 
curr_dist = []    
for row_idx in range(W.shape[0]):
    curr_vec = W[row_idx,:]
    normalized_dist = (dist(curr_vec,vec_e1))/relTau
    curr_dist.append(normalized_dist)


curr_dist[e1_idx] = np.Inf
curr_closest_indices = np.argsort(curr_dist)[:10]
for r_idx in curr_closest_indices:
    relev_syn = IDsToSyns_f[r_idx]
    print(curr_dist[r_idx], relev_syn.name(), relev_syn.definition())



0.999999957925 non-euclidean_geometry.n.01 (mathematics) geometry based on axioms different from Euclid's
0.999999990071 plane_geometry.n.01 the geometry of 2-dimensional figures
0.999999994248 solid_geometry.n.01 the geometry of 3-dimensional space
0.999999994509 fractal_geometry.n.01 (mathematics) the geometry of fractals
1.00000000061 pure_mathematics.n.01 the branches of mathematics that study and develop the principles of mathematics for their own sake rather than for their immediate usefulness
1.00000000189 spherical_geometry.n.01 (mathematics) the geometry of figures on the surface of a sphere
1.00000000937 projective_geometry.n.01 the geometry of properties that remain invariant under projection
1.0000000101 analytic_geometry.n.01 the use of algebra to study geometric properties; operates on symbols defined in a coordinate system
1.00000001566 affine_geometry.n.01 the geometry of affine transformations
1.00000002831 elementary_geometry.n.01 (mathematics) geometry based on Eucli

In [21]:
# Word analogy for selected examples.

vector_dim = 10
rel = 'hypernym_demo'
emb_dict_curr = RelEmbDict[rel]
vocab_size = len(emb_dict_curr)
W = np.zeros((vocab_size, vector_dim))
relTau = RelTauDict[rel]


# Choose the entities.
e1 = wn.synset('guitarist.n.01')
e1_idx = SynsToIDs_f[e1]

e2 = wn.synset('musician.n.01')
e2_idx = SynsToIDs_f[e2]

e3 = wn.synset('novelist.n.01')
e3_idx = SynsToIDs_f[e3]


for idx, vec in emb_dict_curr.items():
    W[idx,:] = vec
    
vec_e1 = emb_dict_curr[e1_idx]
vec_e2 = emb_dict_curr[e2_idx]
vec_e3 = emb_dict_curr[e3_idx]


vec1_ = hyp_scale(-1, vec_e1)
left_sum = hyp_weighted_sum(1, 1, vec_e2, vec1_)
vec_search = hyp_weighted_sum(1, 1, left_sum, vec_e3)

curr_dist = []    
for row_idx in range(W.shape[0]):
    curr_vec = W[row_idx,:]
    normalized_dist = (dist(curr_vec, vec_search))/relTau
    curr_dist.append(normalized_dist)


curr_dist[e1_idx] = np.Inf
curr_dist[e2_idx] = np.Inf
curr_dist[e3_idx] = np.Inf

curr_closest_indices = np.argsort(curr_dist)[:10]
for r_idx in curr_closest_indices:
    relev_syn = IDsToSyns_f[r_idx]
    print(curr_dist[r_idx], relev_syn.name(), relev_syn.definition())

       

0.716374331939 writer.n.01 writes (books or stories or articles or the like) professionally (for pay)
0.987304286504 communicator.n.01 a person who communicates with others
1.17175459928 librettist.n.01 author of words to be set to music in an opera or operetta
1.18552618387 announcer.n.01 someone who proclaims a message publicly
1.18607557487 essayist.n.01 a writer of literary works
1.20045196565 commentator.n.02 a writer who reports and analyzes events of the day
1.21288127055 gagman.n.02 someone who writes comic material for public performers
1.21336310414 sympathizer.n.01 commiserates with someone who has had misfortune
1.25372619738 lyricist.n.01 a person who writes the words for songs
1.26142195216 alliterator.n.01 a speaker or writer who makes use of alliteration
